# Análise ENEM 2010

## Carregando Bibliotecas

### Variável de ambiente

In [1]:
import os

os.environ["SPARK_HOME"] = "D:\spark\spark-3.2.1-bin-hadoop3.2"
os.environ['HADOOP_HOME'] = "D:\spark\spark-3.2.1-bin-hadoop3.2\hadoop" 

os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

### Pyspark

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

import pyspark.sql as py

import pyspark.pandas as ps

In [4]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

### Visualização

In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

## Carregando os dados

In [6]:
enem_2010 = ps.read_csv("base_dados/enem/microdados_enem2010/DADOS/MICRODADOS_enem_2010.csv", sep=';')

In [7]:
enem_2010 = enem_2010.rename(columns={  'Q01': 'Q001', 
                                        'Q02' :'Q002',
                                        'Q03': 'Q003',
                                        'Q04': 'Q004',
                                        'Q05': 'Q005',
                                        'Q06': 'Q006'})

In [8]:
columns=['NU_IDADE', 'TP_COR_RACA', 'TP_ESTADO_CIVIL' ,'TP_SEXO', 'TP_ST_CONCLUSAO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006']

In [9]:
enem_2010_selected = enem_2010[columns]

## Verificando Valores Ausentes

### Visualização

In [10]:
import plotly.graph_objects as go

In [11]:
missing = enem_2010_selected.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace = True)

In [12]:
texto_grafico = []
for i in range(len(missing)):
    texto_grafico.append(str(missing.values[i]))

D:\spark\spark-3.2.1-bin-hadoop3.2\python\pyspark\pandas\generic.py:638: UserWarning: We recommend using `Series.to_numpy()` instead.
  warnings.warn("We recommend using `{}.to_numpy()` instead.".format(type(self).__name__))


In [13]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=missing.index.tolist(),
    y=missing.values,
    #mode="markers+text",
    name="Markers and Text",
    text=texto_grafico,
    textposition="outside"
))

fig.update_layout(
    autosize=False,
    width=1000,
    height=500)

fig.update_layout(title_text="Quantidade de Dados Ausentes por Colunas (NAN)",
                  title_font_size=20)

fig.update_xaxes(title_text="Colunas")
fig.update_yaxes(title_text="Quantidade de Valores Ausentes")

fig.show()

D:\spark\spark-3.2.1-bin-hadoop3.2\python\pyspark\pandas\generic.py:638: UserWarning:

We recommend using `Series.to_numpy()` instead.



### Tabela com Porcentagem

In [14]:
var_with_nan = []
def verfify_nan_values(df):
    var_without_nan = []
    var_column = []
    var_percent = []
    total = df.shape[0]
    count = 0
    for index, value in df.isna().sum().iteritems():
        if value == 0:
            var_without_nan.append(index)
        else:
            percent_with_nan = (value / total) * 100
            var_with_nan.append(count)
            var_column.append(index)
            var_percent.append(percent_with_nan)
        count  = count + 1

    d = {'coluna': var_column, 'porcentagem_nan %': var_percent}
    df_nan = ps.DataFrame(data=d)

    return df_nan

In [15]:
df_nan = verfify_nan_values(enem_2010_selected)

df_nan

,coluna,porcentagem_nan %
0,NU_IDADE,0.007177
1,TP_COR_RACA,0.312964
2,TP_ESTADO_CIVIL,0.312964
3,TP_ST_CONCLUSAO,0.000497
4,NU_NOTA_CN,26.833242
5,NU_NOTA_CH,26.833242
6,NU_NOTA_LC,29.613082
7,NU_NOTA_MT,29.613082
8,NU_NOTA_REDACAO,0.000043
9,Q001,0.312964


### Tratamento dos Dados Ausentes

In [16]:
enem_2010_without_nan = enem_2010_selected.dropna(subset = ['NU_NOTA_CN', 'NU_NOTA_CH', 'Q001'])

## Adicionando Média das Notas

Pesquisa com Base curso de Medicina na USP

https://www.sisu.pro.br/sisu/sisu-2021-usp-2.html#:~:text=Pesos%20para%20Medicina&text=Linguagens%20e%20C%C3%B3digos%3A%202%C3%97,Matem%C3%A1tica%3A%203%C3%97500%20%3D%201.500

* Redação: 2
* Linguagens e Códigos: 2
* Ciências Humanas: 2
* Ciências da Natureza: 4
* Matemática: 3

In [17]:
def mean_med(df):
    df['MEDIA_MED'] = ( 
        df['NU_NOTA_CN'] * 4 + 
        df['NU_NOTA_CH'] * 2 + 
        df['NU_NOTA_LC'] * 2 + 
        df['NU_NOTA_MT'] * 3 + 
        df['NU_NOTA_REDACAO'] * 2
    ) / (4+2+2+3+2)

In [18]:
mean_med(enem_2010_without_nan)

Pesquisa com Base curso de Administração na Unicentro Paraná

https://www3.unicentro.br/vestibular/sisu/cursos-e-pesos/


* Redação: 1
* Linguagens e Códigos: 2
* Ciências Humanas: 2
* Ciências da Natureza: 2
* Matemática: 3

In [19]:
def mean_adm(df):
    df['MEDIA_ADM'] = ( 
        df['NU_NOTA_CN'] * 2 + 
        df['NU_NOTA_CH'] * 2 + 
        df['NU_NOTA_LC'] * 2 + 
        df['NU_NOTA_MT'] * 3 + 
        df['NU_NOTA_REDACAO'] * 1
    ) / (2+2+2+3+1)

In [20]:
mean_adm(enem_2010_without_nan)

## Análise dos Agrupamentos

In [21]:
enem_2010_without_nan.columns

Index(['NU_IDADE', 'TP_COR_RACA', 'TP_ESTADO_CIVIL', 'TP_SEXO',
       'TP_ST_CONCLUSAO', 'NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC',
       'NU_NOTA_MT', 'NU_NOTA_REDACAO', 'Q001', 'Q002', 'Q003', 'Q004', 'Q005',
       'Q006', 'MEDIA_MED', 'MEDIA_ADM'],
      dtype='object')

#### Media por Raça

In [22]:
enem_group_raca_media = enem_2010_without_nan[['TP_COR_RACA', 'MEDIA_MED', 'MEDIA_ADM']].groupby(['TP_COR_RACA']).mean()

In [23]:
enem_group_raca_media = enem_group_raca_media.reset_index()

In [24]:
legenda_raca = {
0 : 'Não declarado',
1 : 'Branca',
2 : 'Preta',
3 : 'Parda',
4 : 'Amarela',
5 : 'Indígena',
6 : 'Não dispõe da informação'
}

In [25]:
enem_group_raca_media['TP_COR_RACA'] = enem_group_raca_media['TP_COR_RACA'].map(legenda_raca)

In [26]:
enem_group_raca_media.sort_values(by=['MEDIA_MED'])

,TP_COR_RACA,MEDIA_MED,MEDIA_ADM
2,Indígena,485.826216,485.648034
4,Preta,500.121334,500.060448
1,Parda,502.880740,502.513200
5,Não declarado,519.885988,520.912193
3,Amarela,527.673313,528.430729
0,Branca,534.408718,535.722069


#### Media por Renda

In [27]:
legenda_renda = {
'A':'Nenhuma Renda',
'B':'Até R$ 1.045,00',
'C':'De R$ 1.045,01 até R$ 1.567,50',
'D':'De R$ 1.567,51 até R$ 2.090,00',
'E':'De R$ 2.090,01 até R$ 2.612,50',
'F':'De R$ 2.612,51 até R$ 3.135,00',
'G':'De R$ 3.135,01 até R$ 4.180,00',
'H':'De R$ 4.180,01 até R$ 5.225,00',
'I':'De R$ 5.225,01 até R$ 6.270,00',
'J':'De R$ 6.270,01 até R$ 7.315,00',
'K':'De R$ 7.315,01 até R$ 8.360,00',
'L':'De R$ 8.360,01 até R$ 9.405,00',
'M':'De R$ 9.405,01 até R$ 10.450,00',
'N':'De R$ 10.450,01 até R$ 12.540,00',
'O':'De R$ 12.540,01 até R$ 15.675,00',
'P':'De R$ 15.675,01 até R$ 20.900,00',
'Q':'Acima de R$ 20.900,00'
}

In [28]:
enem_group_renda_media = enem_2010_without_nan[['Q006', 'MEDIA_MED', 'MEDIA_ADM']].groupby(['Q006']).mean()

In [29]:
enem_group_renda_media = enem_group_renda_media.reset_index()

In [30]:
enem_group_renda_media['Q006'] = enem_group_renda_media['Q006'].map(legenda_renda)

In [31]:
enem_group_renda_media.rename(columns={'Q006': 'Faixa Renda'}, inplace=True)

In [32]:
enem_group_renda_media.sort_values(by=['MEDIA_MED'])

,Faixa Renda,MEDIA_MED,MEDIA_ADM
1,"De R$ 1.045,01 até R$ 1.567,50",511.890818,512.528728
2,Nenhuma Renda,517.763258,518.162759
0,"Até R$ 1.045,00",519.790803,520.500253


#### Media por Renda e por Raça

In [33]:
enem_group_renda_raca_media = enem_2010_without_nan[['Q006', 'TP_COR_RACA', 'MEDIA_MED', 'MEDIA_ADM']].groupby(['Q006', 'TP_COR_RACA']).mean()

In [34]:
enem_group_renda_raca_media = enem_group_renda_raca_media.reset_index()

In [35]:
enem_group_renda_raca_media['Q006'] = enem_group_renda_raca_media['Q006'].map(legenda_renda)
enem_group_renda_raca_media['TP_COR_RACA'] = enem_group_renda_raca_media['TP_COR_RACA'].map(legenda_raca)

In [36]:
enem_group_renda_raca_media.rename(columns={'Q006': 'Faixa Renda'}, inplace=True)

In [37]:
enem_group_renda_raca_media.sort_values(by=['MEDIA_MED'])

,Faixa Renda,TP_COR_RACA,MEDIA_MED,MEDIA_ADM
4,Nenhuma Renda,Indígena,483.173717,482.838576
17,"De R$ 1.045,01 até R$ 1.567,50",Indígena,488.823337,489.297794
0,"Até R$ 1.045,00",Indígena,493.263982,493.277440
7,Nenhuma Renda,Preta,498.923301,498.685575
3,Nenhuma Renda,Parda,501.496136,500.963015
11,"De R$ 1.045,01 até R$ 1.567,50",Preta,501.732292,502.141412
1,"De R$ 1.045,01 até R$ 1.567,50",Parda,503.642645,503.712205
16,"Até R$ 1.045,00",Preta,503.686218,504.003098
10,"Até R$ 1.045,00",Parda,507.837663,507.856380
15,"De R$ 1.045,01 até R$ 1.567,50",Amarela,516.086655,516.379251


#### Quantidade por Raça

In [38]:
enem_group_raca_quant = enem_2010_without_nan[['TP_COR_RACA', 'MEDIA_MED']].groupby(['TP_COR_RACA']).count()

In [39]:
enem_group_raca_quant = enem_group_raca_quant.reset_index()

In [40]:
enem_group_raca_quant['TP_COR_RACA'] = enem_group_raca_quant['TP_COR_RACA'].map(legenda_raca)

In [41]:
enem_group_raca_quant.sort_values(by=['MEDIA_MED'])

,TP_COR_RACA,MEDIA_MED
2,Indígena,20590
3,Amarela,71218
5,Não declarado,104761
4,Preta,360968
1,Parda,1243402
0,Branca,1433284


#### Quantidade por Renda e Raça

In [42]:
enem_group_raca_renda_quant = enem_2010_without_nan[['TP_COR_RACA', 'Q006', 'MEDIA_MED']].groupby(['TP_COR_RACA', 'Q006']).count()

In [43]:
enem_group_raca_renda_quant = enem_group_raca_renda_quant.reset_index()

In [44]:
enem_group_raca_renda_quant['TP_COR_RACA'] = enem_group_raca_renda_quant['TP_COR_RACA'].map(legenda_raca)
enem_group_raca_renda_quant['Q006'] = enem_group_raca_renda_quant['Q006'].map(legenda_renda)

In [45]:
enem_group_raca_renda_quant.sort_values(by=['MEDIA_MED'])

,TP_COR_RACA,Q006,MEDIA_MED
14,Indígena,"De R$ 1.045,01 até R$ 1.567,50",2189
13,Indígena,"Até R$ 1.045,00",4187
17,Amarela,"De R$ 1.045,01 até R$ 1.567,50",6689
4,Não declarado,"De R$ 1.045,01 até R$ 1.567,50",11066
7,Amarela,"Até R$ 1.045,00",13861
15,Indígena,Nenhuma Renda,14214
6,Não declarado,"Até R$ 1.045,00",20395
1,Preta,"De R$ 1.045,01 até R$ 1.567,50",42150
16,Amarela,Nenhuma Renda,50668
9,Preta,"Até R$ 1.045,00",65937
